In [ ]:
# 1: Instalación de dependencias
!pip install --upgrade diffusers[torch]==0.27.0 transformers accelerate safetensors
!pip install gradio
!pip install git+https://github.com/huggingface/peft.git
!pip install pytorch-fid

In [ ]:
# 2: Importar librerías
import torch
from diffusers import StableDiffusionPipeline
import gradio as gr
from PIL import Image
import os
import glob

In [ ]:
# 3: Configuración de rutas y modelo
MODEL_ID = 'runwayml/stable-diffusion-v1-5'
OUTPUT_DIR = '/content/gen'
os.makedirs(OUTPUT_DIR, exist_ok=True)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
pipe = StableDiffusionPipeline.from_pretrained(
    MODEL_ID,
    torch_dtype=torch.float16 if device=='cuda' else torch.float32
)
pipe = pipe.to(device)
pipe.safety_checker = None  # opcional

In [ ]:
# 4: Generación de imagen de prueba
prompt = "Infografía educativa de célula vegetal con etiquetas, fondo blanco, estilo claro y didáctico"
with torch.autocast(device):
    img = pipe(prompt, guidance_scale=7.5, num_inference_steps=30).images[0]
img.save(os.path.join(OUTPUT_DIR, 'ejemplo_celula.png'))
img.show()

In [ ]:
# 5: Generación de varias imágenes con prompts distintos
prompts = [
    "Infografía de célula animal señalando núcleo, membrana, citoplasma, fondo blanco, estilo educativo",
    "Diagrama del ciclo del agua con flechas y etiquetas, estilo infografía, fondo claro",
    "Número lineal con fracciones marcadas, diseño imprimible, estilo educativo"
]

for i, p in enumerate(prompts):
    img = pipe(p, guidance_scale=7.5, num_inference_steps=30).images[0]
    img.save(os.path.join(OUTPUT_DIR, f'gen_{i}.png'))

In [ ]:
# 6: Mostrar imágenes generadas
gen_files = glob.glob(os.path.join(OUTPUT_DIR, '*.png'))
for f in gen_files:
    display(Image.open(f))


In [ ]:
# 7: Interfaz Gradio para demo interactiva
def generar(prompt, style, steps, scale):
    full_prompt = f"{prompt}, estilo: {style}, clean vector illustration, educational infographic, clear labels"
    image = pipe(full_prompt, guidance_scale=float(scale), num_inference_steps=int(steps)).images[0]
    return image

demo = gr.Interface(
    fn=generar,
    inputs=[
        gr.Textbox(lines=2, placeholder="Prompt..."),
        gr.Dropdown(['Infografía', 'Ilustración', 'Dibujo escolar', 'Realista suave'], value='Infografía'),
        gr.Slider(10, 50, value=30, step=1, label="Steps"),
        gr.Slider(1, 15, value=7.5, step=0.1, label="Guidance Scale")
    ],
    outputs='image',
    title='EduDiff - Generador de Imágenes Educativas',
    description='Genera imágenes para material didáctico (demo).'
)

demo.launch(share=True)